## Qualidade e limpeza dos Dados

Como primeiro passo, vamos verificar o formato e a qualidade dos dados para modelagem.

In [1]:
# Quantidade de Linhas
!wc -l winequality.csv

6497 winequality.csv


In [2]:
!head -n 2 winequality.csv

Aparentemente, os dados estão utilizando '.' como separador de decimal e ';' como separador de campo

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('winequality.csv', sep = ';')

Vamos verificar os Tipos dos dados

In [5]:
[(col, str(data[col].dtype)) for col in data.columns]

[('type', 'object'),
 ('fixed acidity', 'float64'),
 ('volatile acidity', 'float64'),
 ('citric acid', 'float64'),
 ('residual sugar', 'float64'),
 ('chlorides', 'float64'),
 ('free sulfur dioxide', 'float64'),
 ('total sulfur dioxide', 'float64'),
 ('density', 'float64'),
 ('pH', 'float64'),
 ('sulphates', 'float64'),
 ('alcohol', 'object'),
 ('quality', 'int64')]

A coluna `alcohol` estranhamente está com o tipo de dados `object`, o que me faz suspeitar que o pandas esteja interpretando o campo como uma string. Vamos investigar um pouco

In [6]:
data['alcohol'].unique()

array(['8.8', '9.5', '10.1', '9.9', '9.6', '11', '12', '9.7', '10.8',
       '12.4', '11.4', '12.8', '11.3', '10.5', '9.3', '10', '10.4', '11.6',
       '12.3', '10.2', '9', '11.2', '8.6', '9.4', '9.8', '11.7', '10.9',
       '9.1', '8.9', '10.3', '12.6', '10.7', '12.7', '10.6', '9.2', '8.7',
       '11.5', '11.8', '12.1', '11.1', '8.5', '12.5', '11.9', '12.2',
       '12.9', '13.9', '14', '13.5', '13.3', '13.2', '13.7', '13.4', '13',
       '8', '13.1', '8.0', '13.6', '8.4', '14.2', '11.94',
       '128.933.333.333.333', '114.666.666.666.667', '10.98',
       '100.333.333.333.333', '114.333.333.333.333', '105.333.333.333.333',
       '953.333.333.333.333', '109.333.333.333.333', '113.666.666.666.667',
       '113.333.333.333.333', '110.666.666.666.667', '973.333.333.333.333',
       '11.05', '9.75', '11.35', '9.55', '10.55', '11.45', '14.05',
       '123.333.333.333.333', '12.75', '13.8', '12.15', '13.05',
       '112.666.666.666.667', '105.666.666.666.667', '117.333.333.333.333',
   

A variável parece conter valores estranhos, como '135.666.666.666.667'. Nesse caso, suspeitaria de algum problema no processo de criação da base e questionaria sobre o problema. No entanto, vou verificar se é possível prosseguir com a modelagem com estes dados.

Temos tres possíveis soluções:

1. Não utilizar a variável `alcohol`
2. Substituir os valores estranhos por valores 'missing'
3. Excluir as linhas nas quais estes valores ocorrem.

A primeira solução me parece desnecessária, pois implicaria na não utilização de uma variável potencialmente importante, porém dependendo do tamanho do 'estrago', pode ser a mais indicada. 

Já a segunda, por envolver valores missings, poderia nos causar problemas com certos tipos de modelos e nos obrigaria utilizar algum tipo de imputação. 

A terceira solução implicaria na perda de alguns dados, o que pode ser ruim dependendo da quantidade dos mesmos.

Para decidir entre as soluções, vamos ver quantas observações apresentam tais valores estranhos.

Creio que a maneira mais fácil de fazer isso, seja através da utilização de expressões regulares e o método `contains` do Pandas.

In [7]:
probl = data[data['alcohol'].str.contains(r'\d{3}\.\d{3}\.\d{3}\.\d{3}')]

In [8]:
probl['alcohol'].unique()

array(['128.933.333.333.333', '114.666.666.666.667', '100.333.333.333.333',
       '114.333.333.333.333', '105.333.333.333.333', '953.333.333.333.333',
       '109.333.333.333.333', '113.666.666.666.667', '113.333.333.333.333',
       '110.666.666.666.667', '973.333.333.333.333', '123.333.333.333.333',
       '112.666.666.666.667', '105.666.666.666.667', '117.333.333.333.333',
       '109.666.666.666.667', '101.333.333.333.333', '104.666.666.666.667',
       '116.333.333.333.333', '131.333.333.333.333', '120.666.666.666.667',
       '963.333.333.333.333', '956.666.666.666.667', '135.666.666.666.667',
       '923.333.333.333.333'], dtype=object)

In [9]:
len(probl)

40

Como se tratam de apenas 40 observações de 6000+, creio que excluí-las não afetará a nossa análise.

In [10]:
clean_data = data[~data['alcohol'].str.contains(r'\d{3}\.\d{3}\.\d{3}\.\d{3}')].copy()

In [11]:
assert(len(clean_data) + len(probl) == len(data))

In [12]:
clean_data['alcohol'] = clean_data['alcohol'].astype('float64')

In [13]:
pd.options.display.float_format = '{:,.3f}'.format

In [14]:
clean_data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,"6,457.000","6,457.000","6,457.000","6,457.000","6,457.000","6,457.000","6,457.000","6,457.000","6,457.000","6,457.000","6,457.000","6,457.000"
mean,7.219,0.340,0.319,5.446,0.056,30.496,115.691,1.715,3.219,0.531,10.489,5.817
std,1.295,0.165,0.146,4.764,0.035,17.757,56.595,7.659,0.161,0.149,1.193,0.874
min,3.800,0.080,0.000,0.600,0.009,1.000,6.000,0.987,2.720,0.220,8.000,3.000
25%,6.400,0.230,0.250,1.800,0.038,17.000,77.000,0.992,3.110,0.430,9.500,5.000
50%,7.000,0.290,0.310,3.000,0.047,29.000,118.000,0.995,3.210,0.510,10.300,6.000
75%,7.700,0.400,0.390,8.100,0.065,41.000,156.000,0.997,3.320,0.600,11.300,6.000
max,15.900,1.580,1.660,65.800,0.611,289.000,440.000,103.898,4.010,2.000,14.900,9.000


### Dados Missing

Agora podemos verificar se existem variáveis com valores missing nas nossas observações.

In [15]:
clean_data.isnull().sum().sum()

0

Nossos não apresentam nenhuma observação com valores nulos, logo não é necessário tratamento.

In [16]:
# Salvando os dados para uso posterior

clean_data.to_csv('winequality_clean.csv')